In [ ]:
import matplotlib.pyplot as plt
from __future__ import division

import numpy as np
import cv2

def get_edge(frame):
    kerneld = np.ones((5,5),np.uint8)
    kernele = np.ones((3,3),np.uint8)
    edges = cv2.Canny(frame,150,150)
    edges = cv2.dilate(edges,kerneld,iterations = 1) #5
    edges = cv2.erode(edges,kernele,iterations = 1) #5
    #h, w = edges.shape
    #cv2.line(edges, (0,h-1), (w-1,h-1), (255, 255, 255), 2)
    return edges

def get_mask(frame):
    mask = cv2.adaptiveThreshold(frame,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    mask = cv2.medianBlur(mask,5)
    return mask

def get_moments(obj):
    moments = cv2.moments(obj)
    area = moments['m00']
    cX = 0
    cY = 0
    if (area != 0):
        cX = int(moments['m10']/area)
        cY = int(moments['m01']/area)
    return cX, cY

def draw_contours(obj, img):
    x, y = get_moments(obj)
    if ((x!=0) and (y!=0)):
        contours, hierarchy = cv2.findContours(obj.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(img, contours, -1, (0, 0, 255), 20)
    return img

def fill_contours(frame):
    cv2.fillPoly(frame, pts =[contours], color=(255,255,255))
    return frame

def get_circles(frame):
    circles = cv2.HoughCircles(frame, cv2.cv.CV_HOUGH_GRADIENT,1,20, param1=50,param2=30,minRadius=0,maxRadius=0)
    circles = np.uint16(np.around(circles))
    for i in circles[0,:]:
        # draw the outer circle
        cv2.circle(frame,(i[0],i[1]),i[2],(0,255,0),2)
        # draw the center of the circle
        cv2.circle(frame,(i[0],i[1]),2,(0,0,255),3)
    return frame

cap = cv2.VideoCapture('video/sumo_red.mp4')

while(cap.isOpened()):
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    h,w = gray.shape
    reduce_frame = gray[int(1.5*h/4):h,0:w]
    
    #mask = get_mask(gray)
    
    edge = get_edge(reduce_frame)
    cv2.imshow('edge',edge)
    
    mask =draw_contours(edge, reduce_frame)
    cv2.imshow('mask',mask)
    
    
#    filled = fill_contours(edge)
#    cv2.imshow('filled',filled)
    #circles = get_circles(edge)
    #cv2.imshow('circles',circles)
    
    cv2.imshow('frame',reduce_frame)
    cv2.imshow('frame',frame)
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()